# Fraud detection

In [65]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

In [66]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import recall_score,f1_score,precision_score
#Visualization
import matplotlib as mpl
import seaborn as sns
from sklearn.metrics import auc
from sklearn import metrics
from sklearn.metrics import roc_curve,roc_auc_score


In [67]:
from scipy.io.arff import loadarff 
from sklearn.preprocessing import LabelEncoder
raw_data = loadarff('base_validacao_02.arff')
data = pd.DataFrame(raw_data[0])
data.rename(columns={'flag_fraude_cat': 'Class'}, inplace=True)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Label encode the 'Class' column
data['Class'] = label_encoder.fit_transform(data['Class'])
tr_data = data
X = tr_data.drop(['Class'], axis = 'columns')
Label_Data = tr_data['Class']
y=tr_data['Class']
X_test=X
y_test=y

In [68]:
raw_data = loadarff('base_desenv_02.arff')
df = pd.DataFrame(raw_data[0])
df.rename(columns={'flag_fraude_cat': 'Class'}, inplace=True)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Label encode the 'Class' column
df['Class'] = label_encoder.fit_transform(df['Class'])

In [69]:
df.describe()

,mcc_cat,mcc_ant_cat,cep_cat,cep_ant_cat,valor_trans_cat,valor_trans_ant_cat,pos_entry,lim_cred_cat,bandeira_cat,variante_cat,score_cat,tp_pessoa_cat,trans_nacional_cat,qtde_parc_cat,velocidade_cat,dif_score_cat,trans_limit_cat,Class
count,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000,29104.000000
mean,10.900667,6.598612,3.907264,2.284394,4.977220,3.351979,3.835899,5.888503,2.351567,2.015049,0.631941,0.982545,0.005154,1.211792,3.361291,2.933308,0.069269,0.037521
std,6.261320,7.303525,2.442654,2.653015,2.432682,2.714225,3.160507,2.436714,1.719125,0.834260,1.703692,0.130960,0.071607,0.469841,1.709276,0.983518,0.397668,0.190037
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,6.000000,0.000000,2.000000,0.000000,3.000000,1.000000,2.000000,4.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000,3.000000,0.000000,0.000000
50%,9.000000,6.000000,4.000000,1.000000,5.000000,2.000000,2.000000,6.000000,1.000000,2.000000,0.000000,1.000000,0.000000,1.000000,2.000000,3.000000,0.000000,0.000000
75%,15.000000,12.000000,6.000000,4.000000,7.000000,5.000000,9.000000,8.000000,5.000000,3.000000,0.000000,1.000000,0.000000,1.000000,5.000000,3.000000,0.000000,0.000000
max,32.000000,32.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,5.000000,5.000000,9.000000,1.000000,1.000000,3.000000,7.000000,5.000000,8.000000,1.000000


In [70]:
df.shape

(29104, 18)

In [71]:
df1=df
Target=df['Class']
df.drop('Class',axis=1, inplace=True)

Final Dataframe

# outlier

## OneClassSVM

In [72]:
from sklearn.svm import OneClassSVM

In [73]:
df1_fraud=df1[Target==1]
df1_normal=df1[Target==0]
df1_fraud.shape,df1_normal.shape

((1092, 17), (28012, 17))

In [74]:
clf = OneClassSVM()
res = clf.fit_predict(df1_fraud)
np.unique(res)

array([-1,  1], dtype=int64)

In [75]:
df1_normal['Class']=0
df1_fraud['Class']=res

In [76]:
frames = [df1_normal, df1_fraud]
result = pd.concat(frames)

In [77]:
osvm_res=result[result['Class']!=-1]
osvm_out=result[result['Class']==-1]
result[result['Class']==-1].shape

(541, 18)

In [78]:
osvm_t=pd.DataFrame(osvm_res['Class'])
osvm_res.drop(['Class'],axis=1,inplace=True)
osvm_ot=pd.DataFrame(osvm_out['Class'])
osvm_out.drop(['Class'],axis=1,inplace=True)

# Balancing

In [79]:
from imblearn.over_sampling import SMOTE 
from sklearn.metrics import accuracy_score
from statistics import mean
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay

X_train=osvm_res.copy()
X_train.reset_index(drop=True,inplace=True)
y_train=osvm_t.copy()
y_train.reset_index(drop=True,inplace=True)

In [80]:
ros = SMOTE(random_state=0,sampling_strategy=0.0526315789)
X_resampled, y_resampled = ros.fit_resample(X_train,y_train)
X_resampled=pd.concat([X_resampled, osvm_out])
y_resampled=pd.concat([y_resampled, osvm_ot])
y_resampled[y_resampled==-1]=1
clf = LGBMClassifier()
clf.fit(X_resampled,y_resampled)
yhat_probs = clf.predict(X_test)

In [81]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_probs)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_probs)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_probs)
print('Recall: %f' % recall)
f1 = f1_score(y_test, yhat_probs)
print('f1: %f' % f1)
auc_score=roc_auc_score(y_test, yhat_probs)
print('auc_score: %f' % auc_score)

Accuracy: 0.969784
Precision: 0.686441
Recall: 0.346895
f1: 0.460882
auc_score: 0.670384
